In [1]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2023-10-20 16:02:48.375289: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 16:02:48.972144: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-20 16:02:49.577884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 16:02:49.605822: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [3]:
tf.keras.backend.clear_session()

In [4]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [5]:
data = pd.read_csv('2003_mo_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Wind Gust (m/s),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso Do Sul - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,3.127511,951.776390,...,3.117091,0.741626,53.166560,8.248824,0.282837,4.562945e+07,2.151917e+06,16.203179,3.957545e+07,33.744
1,2003-2,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,3.124972,952.492950,...,3.134253,0.741710,53.245714,8.242935,0.284487,4.568706e+07,2.153542e+06,16.206131,3.959155e+07,32.230
2,2003-3,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,2.985641,953.076971,...,3.225807,0.741794,52.994640,8.237046,0.285379,4.574467e+07,2.155168e+06,16.209084,3.960765e+07,33.559
3,2003-4,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,2.615077,953.978721,...,3.365984,0.741878,53.019283,8.231157,0.286089,4.580228e+07,2.156794e+06,16.212036,3.962375e+07,34.283
4,2003-5,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,2.488609,954.723605,...,3.546947,0.741962,53.539593,8.225268,0.286122,4.585990e+07,2.158419e+06,16.214989,3.963985e+07,36.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2002-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.658799,953.010197,...,4.247241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,2002-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.641716,952.337589,...,4.169688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,2002-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.858744,951.610669,...,3.862138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,2002-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.964151,951.513490,...,3.498752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
0,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,1.252955,-0.808277,0.266092,...,1.197653,-0.449229,-1.739598,-1.432541,1.216541,-0.459341,-1.637645,-2.402906,-1.784438,-1.738619
1,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,1.247299,-0.233865,0.313424,...,0.875708,-0.404380,-1.720624,-1.423955,1.206220,-0.422027,-1.622679,-2.361122,-1.774070,-1.724323
2,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,0.937052,0.234300,0.055278,...,0.462888,-0.165113,-1.701651,-1.451188,1.195898,-0.401858,-1.607713,-2.319339,-1.763703,-1.710027
3,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,0.111919,0.957165,-0.140581,...,-0.250400,0.201227,-1.682677,-1.448515,1.185577,-0.385815,-1.592747,-2.277555,-1.753336,-1.695732
4,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-0.169687,1.554281,-0.179669,...,-1.024587,0.674153,-1.663704,-1.392079,1.175256,-0.385067,-1.577781,-2.235771,-1.742968,-1.681436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-2.553933,0.707467,-3.436778,...,-0.325065,-0.424571,1.190288,-0.831849,-1.205180,0.316446,1.171465,-0.292680,0.923349,1.021439
188,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,-2.799957,0.017175,-3.869271,...,0.007661,-0.794890,1.165389,-0.799920,-1.216425,0.383902,1.161163,-0.292842,0.905256,1.007593
189,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,-2.501907,-0.193357,-3.801971,...,0.406195,-1.106080,1.140489,-0.711338,-1.227670,0.452548,1.150861,-0.293004,0.887162,0.993747
190,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,-1.813399,-0.411523,-3.282035,...,0.656710,-1.448804,1.115590,-0.636591,-1.238915,0.528940,1.140559,-0.293166,0.869069,0.979902


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
247       NaN
248       NaN
249       NaN
250       NaN
251       NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 252, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
0,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,1.252955,-0.808277,0.266092,...,1.197653,-0.449229,-1.739598,-1.432541,1.216541,-0.459341,-1.637645,-2.402906,-1.784438,-1.738619
1,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,1.247299,-0.233865,0.313424,...,0.875708,-0.404380,-1.720624,-1.423955,1.206220,-0.422027,-1.622679,-2.361122,-1.774070,-1.724323
2,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,0.937052,0.234300,0.055278,...,0.462888,-0.165113,-1.701651,-1.451188,1.195898,-0.401858,-1.607713,-2.319339,-1.763703,-1.710027
3,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,0.111919,0.957165,-0.140581,...,-0.250400,0.201227,-1.682677,-1.448515,1.185577,-0.385815,-1.592747,-2.277555,-1.753336,-1.695732
4,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-0.169687,1.554281,-0.179669,...,-1.024587,0.674153,-1.663704,-1.392079,1.175256,-0.385067,-1.577781,-2.235771,-1.742968,-1.681436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,0.773256,0.199442,-0.148132,...,1.109508,-1.545830,1.523477,-0.303048,-0.435460,0.586449,1.207622,0.129567,1.062247,1.123802
158,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,0.376208,0.967841,-0.564128,...,0.859989,-1.229690,1.520281,-0.384401,-0.482211,0.549251,1.211468,0.098041,1.062771,1.125391
159,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,-0.186458,1.481082,-0.861031,...,0.272962,-0.840302,1.517085,-0.431034,-0.528962,0.512196,1.215313,0.066515,1.063296,1.126980
160,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,-0.460993,1.935798,-1.230789,...,-0.089679,-0.405728,1.513890,-0.493901,-0.575713,0.470611,1.219159,0.034989,1.063820,1.128569


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
157    66.462
158    79.514
159    63.281
160    77.205
161    71.168
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [11]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [12]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
126,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,1.042425,1.378471,0.050176,...,-1.273459,0.510865,0.553261,1.201269,-1.145074,1.284598,0.792487,0.828557,0.883359,0.868222
127,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,1.150081,0.690596,0.638381,...,-1.322725,0.410286,0.566483,1.181208,-1.093669,1.294211,0.812167,0.824916,0.896439,0.883314
128,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,1.154859,-0.152213,0.710317,...,-0.674601,-0.017190,0.579705,1.148281,-1.042264,1.299852,0.831848,0.821275,0.909518,0.898405
129,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,1.113955,-0.516934,1.036375,...,0.011451,-0.628063,0.592927,1.106818,-0.990860,1.306641,0.851528,0.817634,0.922598,0.913496
130,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,0.765552,-0.921730,1.060716,...,0.414175,-1.026418,0.606148,1.062051,-0.939455,1.314465,0.871209,0.813993,0.935678,0.928588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-2.553933,0.707467,-3.436778,...,-0.325065,-0.424571,1.190288,-0.831849,-1.205180,0.316446,1.171465,-0.292680,0.923349,1.021439
188,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,-2.799957,0.017175,-3.869271,...,0.007661,-0.794890,1.165389,-0.799920,-1.216425,0.383902,1.161163,-0.292842,0.905256,1.007593
189,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,-2.501907,-0.193357,-3.801971,...,0.406195,-1.106080,1.140489,-0.711338,-1.227670,0.452548,1.150861,-0.293004,0.887162,0.993747
190,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,-1.813399,-0.411523,-3.282035,...,0.656710,-1.448804,1.115590,-0.636591,-1.238915,0.528940,1.140559,-0.293166,0.869069,0.979902


In [13]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [14]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [15]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1429286229, 4244866440, 2573288303, 1378446365, 2641880074, 1916111494, 3766199734, 2645751276, 671584399, 1415103000, 3003368263, 1188213760, 4137354597, 859924274, 1441765892, 542747980, 1088435087, 3027142062, 160218087, 2818707066]


Step: 0 ___________________________________________


2023-10-20 16:02:49.991413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 16:02:49.991690: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 16:02:49.991903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

val_loss: 36.04767990112305
winner_seed: 1429286229


Step: 1 ___________________________________________
val_loss: 37.857765197753906


Step: 2 ___________________________________________
val_loss: 33.9044075012207
winner_seed: 2573288303


Step: 3 ___________________________________________
val_loss: 39.161746978759766


Step: 4 ___________________________________________
val_loss: 38.720176696777344


Step: 5 ___________________________________________
val_loss: 43.564918518066406


Step: 6 ___________________________________________
val_loss: 30.079389572143555
winner_seed: 3766199734


Step: 7 ___________________________________________
val_loss: 35.76713562011719


Step: 8 ___________________________________________
val_loss: 32.93394470214844


Step: 9 ___________________________________________
val_loss: 38.08293914794922


Step: 10 ___________________________________________
val_loss: 36.31841278076172


Step: 11 ___________________________________________
val_loss: 35.6457138

In [18]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 133ms/step - loss: 6060.9253 - val_loss: 627.7786
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 756.2729 - val_loss: 934.5681
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 1101.4938 - val_loss: 604.2898
Epoch 4/10000
4/4 [==============================] - 0s 12ms/step - loss: 742.0925 - val_loss: 691.5187
Epoch 5/10000
4/4 [==============================] - 0s 11ms/step - loss: 693.2310 - val_loss: 737.8197
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 730.2061 - val_loss: 306.7906
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 326.7924 - val_loss: 337.4343
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 344.1428 - val_loss: 508.3873
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 648.7969 - val_loss: 447.9808
Epoch 10/10000
4/4 [==============================] - 0s 10ms/

4/4 [==============================] - 0s 11ms/step - loss: 22.5525 - val_loss: 57.8349
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7076 - val_loss: 50.0053
Epoch 82/10000
4/4 [==============================] - 0s 13ms/step - loss: 33.0932 - val_loss: 53.0478
Epoch 83/10000
4/4 [==============================] - 0s 11ms/step - loss: 21.6613 - val_loss: 28.8389
Epoch 84/10000
4/4 [==============================] - 0s 12ms/step - loss: 32.4492 - val_loss: 65.4818
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 30.5009 - val_loss: 29.7393
Epoch 86/10000
4/4 [==============================] - 0s 11ms/step - loss: 21.5089 - val_loss: 60.3597
Epoch 87/10000
4/4 [==============================] - 0s 11ms/step - loss: 26.0045 - val_loss: 38.0614
Epoch 88/10000
4/4 [==============================] - 0s 11ms/step - loss: 35.3729 - val_loss: 33.4941
Epoch 89/10000
4/4 [==============================] - 0s 11ms/step - loss: 22.4765 - val

4/4 [==============================] - 0s 10ms/step - loss: 26.1484 - val_loss: 37.4470
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.7036 - val_loss: 28.9550
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 19.3195 - val_loss: 32.9292
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.6139 - val_loss: 32.2461
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 19.9789 - val_loss: 44.1023
Epoch 164/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.5897 - val_loss: 30.9005
Epoch 165/10000
4/4 [==============================] - 0s 11ms/step - loss: 14.6740 - val_loss: 31.3342
Epoch 166/10000
4/4 [==============================] - 0s 11ms/step - loss: 22.0105 - val_loss: 30.5389
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 19.7583 - val_loss: 41.6616
Epoch 168/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.32

Epoch 238/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.9398 - val_loss: 45.8621
Epoch 239/10000
4/4 [==============================] - 0s 11ms/step - loss: 21.0492 - val_loss: 32.6542
Epoch 240/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.1349 - val_loss: 36.5683
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.7258 - val_loss: 30.6635
Epoch 242/10000
4/4 [==============================] - 0s 11ms/step - loss: 14.3740 - val_loss: 32.3602
Epoch 243/10000
4/4 [==============================] - 0s 11ms/step - loss: 14.3364 - val_loss: 35.7379
Epoch 244/10000
4/4 [==============================] - 0s 11ms/step - loss: 15.9293 - val_loss: 33.9179
Epoch 245/10000
4/4 [==============================] - 0s 11ms/step - loss: 15.2791 - val_loss: 31.0870
Epoch 246/10000
4/4 [==============================] - 0s 11ms/step - loss: 15.3557 - val_loss: 30.9091
Epoch 247/10000
4/4 [==============================] - 0s 10ms/s

Epoch 317/10000
4/4 [==============================] - 0s 11ms/step - loss: 14.7467 - val_loss: 29.6748
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.0589 - val_loss: 41.9936
Epoch 319/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.5142 - val_loss: 29.3793
Epoch 320/10000
4/4 [==============================] - 0s 10ms/step - loss: 22.2542 - val_loss: 33.8896
Epoch 321/10000
4/4 [==============================] - 0s 11ms/step - loss: 18.9801 - val_loss: 27.9636
Epoch 322/10000
4/4 [==============================] - 0s 11ms/step - loss: 13.2152 - val_loss: 29.9625
Epoch 323/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.3730 - val_loss: 32.2092
Epoch 324/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.7037 - val_loss: 31.0773
Epoch 325/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.7004 - val_loss: 36.3289
Epoch 326/10000
4/4 [==============================] - 0s 11ms/st

...

4/4 [==============================] - 0s 10ms/step - loss: 10.7518 - val_loss: 36.7638
Epoch 634/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.6853 - val_loss: 34.9768
Epoch 635/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.2584 - val_loss: 33.3263
Epoch 636/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.5045 - val_loss: 34.2136
Epoch 637/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.5547 - val_loss: 31.5815
Epoch 638/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.6866 - val_loss: 31.6839
Epoch 639/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.9342 - val_loss: 33.4978
Epoch 640/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.0566 - val_loss: 34.5412
Epoch 641/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.8479 - val_loss: 35.8879
Epoch 642/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.2420 - val_

Epoch 713/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.5592 - val_loss: 34.8389
Epoch 714/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.5966 - val_loss: 35.8244
Epoch 715/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.2048 - val_loss: 36.1958
Epoch 716/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.4062 - val_loss: 35.9611
Epoch 717/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.6850 - val_loss: 36.5842
Epoch 718/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.0749 - val_loss: 39.3039
Epoch 719/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.7004 - val_loss: 41.4824
Epoch 720/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.0416 - val_loss: 38.5977
Epoch 721/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.9960 - val_loss: 44.0072
Epoch 722/10000
4/4 [==============================] - 0s 8ms/step - loss: 1

4/4 [==============================] - 0s 9ms/step - loss: 10.0942 - val_loss: 37.1729
Epoch 793/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.1555 - val_loss: 38.0300
Epoch 794/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.5194 - val_loss: 36.9489
Epoch 795/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.4330 - val_loss: 35.3414
Epoch 796/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.4398 - val_loss: 35.3033
Epoch 797/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.6799 - val_loss: 38.4484
Epoch 798/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.6680 - val_loss: 36.3542
Epoch 799/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.8142 - val_loss: 34.7046
Epoch 800/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.6365 - val_loss: 35.4858
Epoch 801/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.8124 - val_loss: 3

4/4 [==============================] - 0s 10ms/step - loss: 8.9356 - val_loss: 35.1796
Epoch 873/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.1100 - val_loss: 37.9148
Epoch 874/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.7435 - val_loss: 37.1251
Epoch 875/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.9659 - val_loss: 36.7347
Epoch 876/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.2879 - val_loss: 34.7783
Epoch 877/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.3513 - val_loss: 36.5210
Epoch 878/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.4056 - val_loss: 36.1738
Epoch 879/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.1459 - val_loss: 38.4967
Epoch 880/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.0711 - val_loss: 36.1822
Epoch 881/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.5308 - val_loss:

4/4 [==============================] - 0s 9ms/step - loss: 8.2643 - val_loss: 36.7060
Epoch 953/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.5423 - val_loss: 35.0906
Epoch 954/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.0603 - val_loss: 37.2491
Epoch 955/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.3908 - val_loss: 37.5088
Epoch 956/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.3316 - val_loss: 38.6181
Epoch 957/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.2647 - val_loss: 35.9874
Epoch 958/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.0225 - val_loss: 36.2328
Epoch 959/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.2182 - val_loss: 39.6519
Epoch 960/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.7499 - val_loss: 38.2601
Epoch 961/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0460 - val_loss: 36

In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 17ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,82.430878,84.534195,84.964142,83.379448,77.874367,71.994667,68.382263,65.839355,63.88266,62.218376,60.929001,62.236835,66.075699,70.322456,73.81517,75.014236,71.571091,67.448563,63.824444,61.732018,60.548737,60.460667,63.854397,69.007645,74.093964,77.840218,80.202621,80.322952,79.430237,78.562935
Target,80.517,81.348,76.509,71.851,71.187,58.02,63.194,60.58,69.448,74.472,63.776,81.732,83.858,87.863,69.723,72.765,77.937,64.332,77.048,65.66,65.445,78.095,80.998,69.415,87.346,82.888,79.287,81.855,77.495,56.612
Error,1.913879,3.186195,8.455139,11.52845,6.68737,13.974667,5.188263,5.259354,5.565338,12.253624,2.847,19.495167,17.782303,17.540543,4.092171,2.249237,6.365906,3.116562,13.223553,3.927986,4.896263,17.634335,17.143604,0.407356,13.252037,5.047783,0.915619,1.532051,1.935234,21.950935


In [21]:
display(mae)
display(mape)

8.312264

0.11340225

In [24]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [25]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[868.6662]] - Target[852.6339999999999]| =  Error: [[16.032227]]; MAPE:[[0.01880318]]
1/1 [==============================] - 0s 19ms/step
Ano-0: |Prediction[[803.6751]] - Target[893.1390000000001]| =  Error: [[89.46387]]; MAPE:[[0.10016792]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[470.4529]] - Target[465.48300000000006]| =  Error: [[4.9699097]]; MAPE:[[0.01067689]]


[array([[16.032227]], dtype=float32),
 array([[89.46387]], dtype=float32),
 array([[4.9699097]], dtype=float32)]

36.822002

0.043215994